In [3]:
from dataset import PrecomputedDataset

In [1]:
import os, random, math
import subprocess
import numpy as np
from tqdm import tqdm
from pathlib import Path
import cv2
import torch
from torchvision import transforms
from PIL import Image
from diffusers.utils import export_to_video
from diffusers.video_processor import VideoProcessor

from ltx_video.models.autoencoders.vae_encode import (
    get_vae_size_scale_factor,
    latent_to_pixel_coords,
    vae_decode,
    # vae_encode,
)

from ltx_video_lora import *
device = "cuda"
dtype = torch.bfloat16
# ------------------- 

vae = load_latent_models()["vae"].to(device, dtype=dtype)

# def _unpack_latents(
#         latents: torch.Tensor, num_frames: int, height: int, width: int, patch_size: int = 1, patch_size_t: int = 1
#     ) -> torch.Tensor:
#     batch_size = latents.size(0)
#     latents = latents.reshape(batch_size, num_frames, height, width, -1, patch_size_t, patch_size, patch_size)
#     latents = latents.permute(0, 4, 1, 5, 2, 6, 3, 7).flatten(6, 7).flatten(4, 5).flatten(2, 3)
#     return latents

def _normalize_latents(
    latents: torch.Tensor, latents_mean: torch.Tensor, latents_std: torch.Tensor, scaling_factor: float = 1.0,
    reverse=False,
) -> torch.Tensor:
    # Normalize latents across the channel dimension [B, C, F, H, W]
    latents_mean = latents_mean.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    latents_std = latents_std.view(1, -1, 1, 1, 1).to(latents.device, latents.dtype)
    if not reverse:
        latents = (latents - latents_mean) * scaling_factor / latents_std
    else:
        latents = latents * latents_std / scaling_factor + latents_mean
    return latents


dest_dir = "/home/eisneim/www/ml/_video_gen/LTX-Video-3-22/data/images/dataset_extracted"
dataset_dirs = [
    '/media/eisneim/4T/ltx_data_49_blured/game_p11_49x1024x576',
    # '/media/eisneim/4T/ltx_data_49_blured/game_p10_blured_49x1024x576', 
    # '/media/eisneim/4T/ltx_data_49_blured/game_p9_49x1024x576', 
    # '/media/eisneim/4T/ltx_data_49_blured/game_p8_49x1024x576', 
    # '/media/eisneim/4T/ltx_data_49_blured/3dgs_game_1-6_49x1024x576', 
    # '/media/eisneim/4T/ltx_data_49_blured/game_p7_49x1024x576', 
    # "/media/eisneim/4T/ltx_data_81x1024x576/game_p13_81x1024x576",
]


    

/home/eisneim/miniforge3/envs/_learn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-29 14:34:22.171430: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 14:34:22.348508: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-29 14:34:23.694146: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cou

In [5]:
dataset = PrecomputedDataset(dataset_dirs[0])

timestep = torch.tensor([0.05], device=device, dtype=dtype)
# timestep = None
is_video = False


for idx, data in enumerate(dataset):
    if idx > 0:
        break

    _, first_frame, _, _, caption, info = data
    frame = unpack_latents(first_frame.unsqueeze(0).to(device, dtype=dtype), 1, info["height"],  info["width"])
    frame = _normalize_latents(frame, vae.mean_of_means, vae.std_of_means, reverse=True)
    print(frame.shape)
    with torch.no_grad():
        # video =  vae.decode(lt, timestep, return_dict=False)[0]
        image = vae_decode(
            frame, vae, is_video,
            vae_per_channel_normalize=False,
            timestep=timestep,
        )
        print("image decode", image.shape)

load cached videos: 2066
torch.Size([1, 128, 1, 18, 32])
image decode torch.Size([1, 3, 1, 576, 1024])


In [23]:
aa = [279, 1742, 1783, 1041, 957, 2041, 3549]
sum(aa)

11392

In [19]:
import time
str(int(time.time()))

'1743230953'

In [24]:
640/32


20.0

In [25]:
360/32

11.25

In [26]:
11 * 32

352

In [27]:
32 * 21

672

In [28]:
672 / 16 * 9 / 32

11.8125

In [29]:
12 * 32

384

In [44]:
aa = [1742, 279, 1229, 4904]
sum(aa)

8154

In [56]:
[x * 5e-6 for x in range(1, 16)]

[5e-06,
 1e-05,
 1.5000000000000002e-05,
 2e-05,
 2.5e-05,
 3.0000000000000004e-05,
 3.5000000000000004e-05,
 4e-05,
 4.5e-05,
 5e-05,
 5.5e-05,
 6.000000000000001e-05,
 6.500000000000001e-05,
 7.000000000000001e-05,
 7.500000000000001e-05]

In [41]:
9 /3 * 4

12.0

In [43]:
32 * 12

384